In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import pandas as pd
import sys
sys.path.append('../../')
import ML as ml
import glob, os

In [2]:
df = pd.read_csv('earth_testset.csv', sep=',')
df.status[df['status']==-1]=0

/home/kong123/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [3]:
all_features = df.iloc[:,:-1]
all_features

,u,e,a_p,inc
0,0.17,0.04,4.25,0.698132
1,0.13,0.30,2.55,1.745329
2,0.49,0.83,5.01,1.047198
3,0.12,0.62,2.60,2.967060
4,0.46,0.23,1.77,1.047198
...,...,...,...,...
99995,0.06,0.02,1.94,0.000000
99996,0.36,0.83,4.13,1.570796
99997,0.07,0.52,3.53,1.396263
99998,0.26,0.31,2.20,3.141593


In [4]:
s =set(all_features['inc'])
inc = list(s)
inc.sort()
for i in range(len(inc)):
    all_features.inc[all_features['inc']==inc[i]]='%d'%i

In [5]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 标准化后，每个特征的均值变为0，所以可以直接用0来替换缺失值
all_features[numeric_features] = all_features[numeric_features].fillna(0)
#set(all_features['inc'])

In [6]:
all_features = pd.get_dummies(all_features, dummy_na=False)
all_features

,u,e,a_p,inc_0,inc_1,inc_10,inc_11,inc_12,inc_13,inc_14,...,inc_17,inc_18,inc_2,inc_3,inc_4,inc_5,inc_6,inc_7,inc_8,inc_9
0,-0.589159,-1.579121,0.381440,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,-0.866340,-0.676598,-0.923107,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.628288,1.163162,0.964649,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,-0.935635,0.434201,-0.884738,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1.420402,-0.919585,-1.521664,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-1.351407,-1.648546,-1.391210,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,0.727450,1.163162,0.289354,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
99997,-1.282111,0.087076,-0.171074,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
99998,0.034498,-0.641885,-1.191691,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
test_features = torch.tensor(all_features.values, dtype=torch.float)
test_labels = torch.tensor(df.status.values, dtype=torch.long)
batch_size = 512
testset = torch.utils.data.TensorDataset(test_features, test_labels)
test_iter = torch.utils.data.DataLoader(testset, batch_size, shuffle=True)

In [8]:
PATH = "./net.pt"
model = torch.load(PATH)

In [9]:
test_acc = ml.eva_acc(test_iter, model)
test_acc

0.98192

In [10]:
ml.dnn_results(test_iter, model) #将dnn预测的结果导出

196

In [11]:
#读取结果
path = './'
file = glob.glob(os.path.join(path, "df**.csv"))
#合并结果
dl = []
for f in file:
    dl.append(pd.read_csv(f,))
df = pd.concat(dl)
df

,u,e,a_p,predictions,labels
0,0.380974,-0.017061,0.151226,1,1
1,-0.658454,-0.329473,-1.199364,0,0
2,-0.311978,-0.364186,-1.629098,0,0
3,-1.697883,-1.336134,1.133473,1,1
4,1.558993,0.781325,0.887911,1,1
...,...,...,...,...,...
507,-0.589159,-0.051774,0.557937,1,1
508,0.588860,0.746613,0.864890,1,1
509,1.073926,-1.197284,-0.577786,1,1
510,-1.074226,0.781325,-1.352841,0,0


In [12]:
#保存
df.to_csv('results.csv', index=False)
#删除多余的文件
for f in file:
    os.remove(f)

In [13]:
#模型评估
ml.dnn_eva(df)

Accuracy: 0.9819, Precision: 0.9831, Recall: 0.9872, F_1: 0.9851


(59929, 776, 1032, 38263)